# Output In The Construction Industry

-----

### Requirements

We're looking to transform **tabs 1 and 2 only**.

#### Observations & Dimensions

The `observations` should be apparent.

The required dimensions are:

* **Geography** - it's all UK level data (the code for UK is "K02000001")
* **Time** - either a simple year, or a year followed by a quarter, i.e "2002 Q2"
* **Adjustment** - either seasonal or non-seasonal
* **Business** - concatenation of rows 6-9 around hyphens. i.e 'Other New Work - Excluding Infrastructure - Private Commercial'
* **CDID** - ONS specific 4 letter codes. Typically on row 10 

-----
Notes:

* Getting the **Business** dimension cleanly is going to be tricky (read - cant see an obvious way to do it), I'd perhaps leave this one until last.
* It's always worth getting the file out of /sources and having a look over.
* You can't really take CDID as a dimension (dimension items needs to be repeating, not unqiue), it's a good exercise though as if doing this for real we'd likely be taking it as meta/supporting data.

In [1]:
%cd mock-transformations/

/workspace/mock-transformations


In [39]:
from databaker.framework import *
import pandas as pd
import numpy as np

tabs = loadxlstabs("./sources/OIC.xls") # load tabs

Loading ./sources/OIC.xls which has size 759808 bytes
Table names: ['Cover Sheet', 'Contents', 'Table 1a', 'Table 1b', 'Table 2a', 'Table 2b', 'Table 3a', 'Table 3b', 'Table 3c', 'Table 3d', 'Table 4', 'Table 4a', 'Table 5a', 'Table 5b', 'Table 6a', 'Table 7', 'Table 8', 'Table 9', 'Table 10', 'Table 11']


# Thar be dragons
The hierarchy associated with the Business sector and the CDID should be treated as a seperate transform, and then joined together at the end. This will make the transform simpler.

## Transform steps
1. Process headers as a dataset with the CDIDs as observations
2. Process observations as a dataset with the CDIDs as a dimension
3. Merge headers and observations on the CDIDs to create a new complete dataframe

### Headers/Observations
|       | Column A | Column B      | Column C  | Column D      |  Column E     |   Column F    |
|-------|----------|---------------|-----------|---------------|---------------|---------------|
| Row 1 | YYYY MMM | Category A    |           | Category B    |               | Category C    |
| Row 2 | YYYY MMM | Subcategory 1 |           | Subcategory 2 | Subcategory 3 | Subcategory 4 |
| Row 3 | YYYY MMM | Detail i      | Detail ii | Detail iii    | Detail iv     | Detail v      |
| Row 4 | YYYY MMM | Code 1        | Code 2    | Code 3        | Code 4        | Code 5        |
| Row 5 | YYYY MMM | OBS v         | OBS w     | OBS x         | OBS y         | OBS z         |

* Codes 1 through 5 should be treated as the observations, and the Categories, Subcategories, Details should be treated as a dimension
* The acutal Observations (Row 5 and below) should be lined with the CDID (e.g. Codes 1-5) and the dates.
* Provide up-to three levels of business classification dimentions

## Other notes
* Units should be confirmed that these values are indexed to 2016=100.

In [182]:
def fetch_headers(tab) -> pd.DataFrame:
    # We're futorunate that all the header information is the same
    business_level1 = tab.excel_ref("C6").expand(RIGHT)
    business_level2 = tab.excel_ref("C7").expand(RIGHT)
    business_level3 = tab.excel_ref("C8").expand(RIGHT)

    # These are the CDIDs
    observations = tab.excel_ref("C10").expand(RIGHT).is_not_blank()

    dimensions = [HDim(business_level1, "business_level1", CLOSEST, LEFT),
                  HDim(business_level2, "business_level2", CLOSEST, LEFT),
                  HDim(business_level3, "business_level3", CLOSEST, LEFT)]
    
    # Convert to dataframe
    headers_and_codes = ConversionSegment(tab, dimensions, observations).topandas()
    # Convert empty strings to nans
    headers_and_codes = headers_and_codes.replace(r'', np.nan)
    # Transpose
    headers_only = headers_and_codes.transpose()
    # Omit the codes and fill down on nans
    headers_only = headers_only.iloc[2:].fillna(method='ffill')
    # Transpose again and fill down on nans
    headers_only = headers_only.transpose().fillna(method='ffill')

    # Compare values between business_level2 and busines_level3, clear dupes on latter
    mask = (headers_only.loc[:,'business_level3'] == headers_only.loc[:,'business_level2'])
    headers_only.loc[mask, 'business_level3'] = np.nan

    # Compare values between business_level1 and busines_level2, clear dupes on latter
    mask = (headers_only.loc[:,'business_level2'] == headers_only.loc[:,'business_level1'])
    headers_only.loc[mask, 'business_level2'] = np.nan

    # Join
    headers_and_codes = pd.concat([headers_only, headers_and_codes.iloc[:,1]], axis=1)
    return headers_and_codes
    # This is likely broken as the levels aren't closest left. Check the output
    

In [177]:
def fetch_contents(tab) -> pd.DataFrame:
    # Start of the CDIDs is the same throughout all the tabs
    cdids = tab.excel_ref("C10").expand(RIGHT).is_not_blank()

    # There are up-to two levels of the time dimension
    time_level1 = tab.excel_ref("A11").expand(DOWN).is_not_blank()
    time_level2 = tab.excel_ref("B11").expand(DOWN)

    # Put the 🍁 syrup away, no waffle here sadly
    observations = tab.excel_ref("C11").expand(DOWN).expand(RIGHT).is_not_blank()

    dimensions = [HDim(time_level1, "time_level1", CLOSEST, ABOVE),
                  HDim(time_level2, "time_level2", DIRECTLY, LEFT),
                  HDim(cdids, "CDIDs", DIRECTLY, ABOVE),
                  HDimConst("units", tab.excel_ref("P5").value)]
    
    return ConversionSegment(tab, dimensions, observations).topandas()

In [184]:
fetch_contents(tabs[3]), fetch_headers(tabs[3])

(        OBS time_level1 time_level2 CDIDs     units
 0      38.9      1997.0              MV3J  2016=100
 1      52.7      1997.0              MV3K  2016=100
 2      50.8      1997.0              MVL8  2016=100
 3      69.0      1997.0              MV3L  2016=100
 4      49.3      1997.0              MV3M  2016=100
 ...     ...         ...         ...   ...       ...
 3173  104.8      2019.0        July  MV3R  2016=100
 3174  103.5      2019.0        July  MV3S  2016=100
 3175  118.9      2019.0        July  MV3T  2016=100
 3176  110.9      2019.0        July  MV3U  2016=100
 3177  113.7      2019.0        July  MV3V  2016=100
 
 [3178 rows x 5 columns],
                business_level1           business_level2     business_level3  \
 0                  New Housing                    Public                 NaN   
 1                  New Housing                   Private                 NaN   
 2                Total Housing                       NaN                 NaN   
 3          

In [ ]:
for tab in tabs:
    if tab.name not in ['Table 1a', 'Table 1b', 'Table 2a', 'Table 2b']:
        continue

    